In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
import re
train = pd.read_csv(r"C:\Users\raven\Downloads\sf-crime\train.csv\train.csv")
test  = pd.read_csv(r"C:\Users\raven\Downloads\sf-crime\test.csv\test.csv")
train1 = train.drop(['Descript', 'Resolution'], axis = 1)

In [2]:
train1 = train1.query('Y != 90')

In [3]:
# Case9 PdDistrictのみone-hot train1.ver

Dt = pd.to_datetime(train1['Dates'])
train1['Date'] = Dt.dt.date
train1['Year'] = Dt.dt.year
train1['Month'] = Dt.dt.month
train1['Day'] = Dt.dt.day
train1['Hour'] = Dt.dt.hour
train1.drop(['Dates'], axis = 1, inplace = True)

le = LabelEncoder()
train1['DayOfWeek'] = le.fit_transform(train1['DayOfWeek'])

Ad_dict = {'.* / .*':0, '.*':1}
train1['Address'] = train1['Address'].replace(Ad_dict,regex = True)

train1['Category'] = le.fit_transform(train1['Category'])

train1_PD = pd.get_dummies(train1['PdDistrict'])

train1 = pd.concat([train1, train1_PD], axis = 1)

train1.drop(['PdDistrict'], axis = 1, inplace=True)

train1.head()

,Category,DayOfWeek,Address,X,Y,Date,Year,Month,Day,Hour,BAYVIEW,CENTRAL,INGLESIDE,MISSION,NORTHERN,PARK,RICHMOND,SOUTHERN,TARAVAL,TENDERLOIN
0,37,6,0,-122.425892,37.774599,2015-05-13,2015,5,13,23,0,0,0,0,1,0,0,0,0,0
1,21,6,0,-122.425892,37.774599,2015-05-13,2015,5,13,23,0,0,0,0,1,0,0,0,0,0
2,21,6,0,-122.424363,37.800414,2015-05-13,2015,5,13,23,0,0,0,0,1,0,0,0,0,0
3,16,6,1,-122.426995,37.800873,2015-05-13,2015,5,13,23,0,0,0,0,1,0,0,0,0,0
4,16,6,1,-122.438738,37.771541,2015-05-13,2015,5,13,23,0,0,0,0,0,1,0,0,0,0


In [4]:
test.head()

,Id,Dates,DayOfWeek,PdDistrict,Address,X,Y
0,0,2015-05-10 23:59:00,Sunday,BAYVIEW,2000 Block of THOMAS AV,-122.399588,37.735051
1,1,2015-05-10 23:51:00,Sunday,BAYVIEW,3RD ST / REVERE AV,-122.391523,37.732432
2,2,2015-05-10 23:50:00,Sunday,NORTHERN,2000 Block of GOUGH ST,-122.426002,37.792212
3,3,2015-05-10 23:45:00,Sunday,INGLESIDE,4700 Block of MISSION ST,-122.437394,37.721412
4,4,2015-05-10 23:45:00,Sunday,INGLESIDE,4700 Block of MISSION ST,-122.437394,37.721412


In [5]:
# Case9 PdDistrictのみone-hot test.ver

Dt = pd.to_datetime(test['Dates'])
test['Date'] = Dt.dt.date
test['Year'] = Dt.dt.year
test['Month'] = Dt.dt.month
test['Day'] = Dt.dt.day
test['Hour'] = Dt.dt.hour
test.drop(['Dates'], axis = 1, inplace = True)

le = LabelEncoder()
train1['DayOfWeek'] = le.fit_transform(train1['DayOfWeek'])

Ad_dict = {'.* / .*':0, '.*':1}
test['Address'] = test['Address'].replace(Ad_dict,regex = True)


test_PD = pd.get_dummies(test['PdDistrict'])

test = pd.concat([test, test_PD], axis = 1)

test.drop(['PdDistrict'], axis = 1, inplace=True)

test.head()

,Id,DayOfWeek,Address,X,Y,Date,Year,Month,Day,Hour,BAYVIEW,CENTRAL,INGLESIDE,MISSION,NORTHERN,PARK,RICHMOND,SOUTHERN,TARAVAL,TENDERLOIN
0,0,Sunday,1,-122.399588,37.735051,2015-05-10,2015,5,10,23,1,0,0,0,0,0,0,0,0,0
1,1,Sunday,0,-122.391523,37.732432,2015-05-10,2015,5,10,23,1,0,0,0,0,0,0,0,0,0
2,2,Sunday,1,-122.426002,37.792212,2015-05-10,2015,5,10,23,0,0,0,0,1,0,0,0,0,0
3,3,Sunday,1,-122.437394,37.721412,2015-05-10,2015,5,10,23,0,0,1,0,0,0,0,0,0,0
4,4,Sunday,1,-122.437394,37.721412,2015-05-10,2015,5,10,23,0,0,1,0,0,0,0,0,0,0
